In [1]:
import numpy as np              # numpy is a general-purpose math package
import yt                       # yt is the data vizualization package
from yt.units import *          # this lets you get things like speed of light in proper units just by typing 'c'
from astropy.table import Table # this is needed for reading in various foggie data byproducts
from astropy.io import ascii    # this too
import matplotlib.pyplot as plt # this is the standard python plotting package, very useful!
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib
import math
from ipynb.fs.defs.Original_frbprojection_code import frb_projection, projection
from scipy import optimize
import pwlf # A library for fitting continuous piecewise linear functions to data (Returns cut-off radius)
import os #for the lacie part 

# These imports are FOGGIE-specific files
from foggie.utils.consistency import *
from foggie.utils.foggie_load import *

#what we need? snap_name, track_name, halo_c_v_name=track_dir + 'halo_c_v', masses_dir=track_dir)

home_dir = '/Users/hylee/foggie/'
laptop_root_dir = home_dir + 'foggie_sims/' # root directory where sims are stored on the laptop
hd_root_dir = '/Volumes/LaCie/' # root directory where sims are stored on the hard drive

root_dir = hd_root_dir # have this as either laptop_root_dir or hd_root_dir depending on where the simulation is stored
code_path = home_dir + 'coding/foggie/foggie/' #path to the cloned github repo

# everything above this is general path, and below this is specific to a given halo
halo = '8508' # which halo
simulation_dir = root_dir + 'halo_00' + halo + '/nref11c_nref9f/' # path where the foggie simulations are stored
output_dir = home_dir + 'foggie_output/halo_00' + halo + '/nref11c_nref9f/' # path where all the outputs will be stored
track_dir = code_path + 'halo_infos/00' + halo + '/nref11c_nref9f/'
track_name = code_path + 'halo_tracks/00' + halo + '/nref11n_selfshield_15/halo_track_200kpc_nref9' #these need to be provided to foggie_load()

In [2]:
def radius_func(od, size, res):
    OD_Qua=[]
    OD_Rad=[]  #OD_Rad & OD_Met HAVE to be defined specifically inside func1/before for loop
    for i in range(70):
        for j in range(70):
            this_quantity_value = od[i][j]
            xindex = i
            yindex = j
            radius = (math.sqrt(((yindex-34)**2+(xindex-34)**2)))*(size/res)
            OD_Qua.append(this_quantity_value)
            OD_Rad.append(radius)

    return OD_Rad, OD_Qua

In [3]:
def binning_func(OD_Rad, OD_Qua, max_radius, min_radius, binsize):
    index = 0 
    quantitylist = []
    sort = np.argsort(OD_Rad)
    sorted_radius = np.array(OD_Rad)[sort]
    sorted_quantity = np.array(OD_Qua)[sort]
    radiuslist = np.arange(min_radius, max_radius, binsize) + binsize/2
    while index < len(radiuslist):
        left_edge = index*binsize
        right_edge = left_edge + binsize 
        new_quantity = np.array(sorted_quantity)[(left_edge <= sorted_radius) & (sorted_radius < right_edge)]
        quantitylist.append((np.mean(new_quantity)))
        index +=1
    return radiuslist, quantitylist

In [4]:
def piecewise_linear(x, x0, y0, k1, k2):
    return np.piecewise(x, [x < x0], [lambda x:k1*x + y0-k1*x0, lambda x:k2*x + y0-k2*x0])

In [ ]:
def plotting(xvalue, yvalue, color, label):
    fig1 = plt.figure()
    plt.plot(xvalue, yvalue, color, label=label)
    plt.legend(loc="upper right", fontsize = 20)
    plt.xlabel("Radius from galactic center [kpc]", fontsize = fontsize)
    plt.ylabel("Density [log 10 g/cm^2]", fontsize = fontsize)
    plt.xticks(fontsize = fontsize/1.5)
    plt.yticks(fontsize = fontsize/1.5)
    plt.xlim([min_radius, max_radius])
    return fig1

In [6]:
DDLIST = []
for file in os.listdir(simulation_dir):
    if file.startswith("DD"):
        DDLIST.append(file)
sorted_dd_list = sorted(DDLIST, key=lambda x: int(x[2:]))

In [7]:
### Changed part

# everything below this is specific to a single snapshot, so below this is where the loop should be


    
snap = 'DD0042' # name of the snapshot output
snap_fullpath = simulation_dir + snap + '/' + snap # full path of where the snapshot output lives

redshiftlist = []
cutofflist = []
pngfile = []

for ~ in sorted_dd_list:
    field = "density"
    axis = "y" 
    width = 200 #[kpc]
    resolution = 70 #[pixel]
    colorlabel = 'Density [log 10 g/cm^2]'
    max_radius = width/2 #[kpc]
    min_radius = 0  #[kpc]
    binsize = 1.5 #[kpc]
    markersize = 10
    fontsize = 20 
    plotlabel = 'Projected along y axis'

    ds, refine_box = foggie_load(snap_fullpath, track_name, halo_c_v_name=track_dir + 'halo_c_v',
                                 disk_relative=False, do_filter_particles=False, masses_dir=track_dir)

    frb = frb_projection(ds, field, axis, refine_box, width, resolution, weight_field = None)

    OD_Rad, OD_Qua = radius_func(np.array(frb["gas",field]), width, resolution)

    radiuslist, quantitylist =  binning_func(OD_Rad, OD_Qua, max_radius, min_radius, binsize)

    p , e = optimize.curve_fit(piecewise_linear, radiuslist, np.log10(quantitylist)) 
    #p for popt (optimization parameter), e for pcov (covariance of popt)
   
    projection_plot = projection(frb, field, width, colorlabel, axis, fontsize = 15)
    segment_plot = plotting(radiuslist, np.log10(quantitylist), 'r-', plotlabel)

    ??plt.savefig(foggie_output + 'DD' + _Density_Profile.jpg')  # => Figure saving directory + file name / size?
    
    redpara = ds.get_parameter("CosmologyCurrentRedshift")
    redshiftlist.append(redpara)
    cutofflist.append(p[0])

'\nDDLIST = []\nfor file in os.listdir(simulation_dir):\n    if file.startswith("DD"):\n        DDLIST.append(file)\n        \n        \n        \n#function here loop over DD files / for loop inside the lacie that prints the DD names. \n\n\n\n\n\n\n\n\n\n# everything below this is specific to a single snapshot, so below this is where the loop should be\n\n\nsnap = \'DD0042\' # name of the snapshot output\nsnap_fullpath = simulation_dir + snap + \'/\' + snap # full path of where the snapshot output lives\n\n\n\n\nredshiftlist = []\ncutofflist = []\npngfile = []\n\nfor DD in filename:\n    field = "density"\n    axis = "z" \n    width = 200 #[kpc]\n    resolution = 70 #[pixel]\n    colorlabel = \'Density [log 10 g/cm^3]\'\n    max_radius = width/2 #[kpc]\n    min_radius = 0  #[kpc]\n    binsize = 1.5 #[kpc]\n    markersize = 10\n    fontsize = 20 \n\n\n    ds, refine_box = foggie_load(snap_fullpath, track_name, halo_c_v_name=track_dir + \'halo_c_v\',\n                                 dis